In [ ]:
!nvidia-smi #v100gpu

Wed Oct 18 05:32:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 모델 로딩

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [ ]:
peft_model_id = "julietz/koalpaca-polyglot-12.8b-law"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
           

Downloading adapter_model.bin 부분 보면 됨

## 모델 추론 함수

In [ ]:
from transformers.generation import GenerationConfig

gen_cfg = GenerationConfig.from_model_config(model.config)
gen_cfg.pad_token_id = tokenizer.eos_token_id

def gen(x):
    gen_cfg.temperature = 0.1
    gen_cfg.max_new_tokens = 256

    gened = model.generate(
        **tokenizer(
            f"User: {x}\n\nAssistant:",
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        do_sample=True,
        generation_config=gen_cfg
    )

    result = tokenizer.decode(gened[0])
    answer = result.split('Assistant:')

    return answer[1].strip()

In [ ]:
gen('한글날은?')

"매년 10월 9일을 말한다.  한글의 우수성을 널리 알리고 세종대왕의 성덕과 위업을 추모하기 위한 기념일이다.  1926년에 음력 9월 29일로 지정된 '가갸날'이 그 시초이며 1928년에 명칭을 '한글날'로 바꾸었다.  1949년에 '관공서의 공휴일에 관한 규정'에 따라 공휴일로 지정되었으며, 2005년에 국경일로 지정되었다.  한글날을 전후한 주간에 정부ㆍ지방자치단체ㆍ민간단체 등이 주관하는 각종 기념행사를 개최한다.  한글날의 제정 및 공포는 일제강점기 이후의 일이지만, 한글 창제를 기념하는 날로서의 한글날은 민족사에서 그 의미가 깊다.  한글날은 훈민정음의 반포일을 기준으로 삼아야 한다는 주장도 있다.   한글날은 1926년에 음력 9월 29일로 지정된 '가갸날'이 그 시초이며 1928년에 명칭을 '한글날'로 바꾸었다.  1949년에 '관공서의 공휴일에 관한 규정'에 따라 공휴일로 지정되"

In [ ]:
gen('080착신과금서비스와 080착신과금사업자에 대해 알려주세요')

'080착신과금서비스는 080번호를 이용하여 전화를 걸면, 전화를 받는 사람이 요금을 부담하는 서비스를 말한다.  080착신과금사업자는 080착신과금서비스를 제공하는 사업자를 말한다.  080착신과금서비스를 제공하는 사업자는 080착신과금서비스를 제공하는 사업자를 말한다.  080착신과금서비스를 제공하는 사업자는 080착신과금서비스를 제공하는 사업자를 말한다.  080착신과금서비스를 제공하는 사업자는 080착신과금서비스를 제공하는 사업자를 말한다.  080착신과금서비스를 제공하는 사업자는 080착신과금서비스를 제공하는 사업자를 말한다.  080착신과금서비스를 제공하는 사업자는 080착신과금서비스를 제공하는 사업자를 말한다.  080착신과금서비스를 제공하는 사업자는 080착신과금서비스를 제공하는 사업자를 말한다.  080착신과금서비스를 제공하는 사업자는 0'

In [ ]:
gen('3종 금융투자업자?')

'다음 각 목의 어느 하나에 해당하는 금융투자업자를 말한다.가. 집합투자업자나. 신탁업자다. 투자매매업자라. 투자중개업자  1) 집합투자업자는 다음 각 목의 어느 하나에 해당하는 자를 말한다.가) 「자본시장과 금융투자업에 관한 법률」 제9조제19항제1호에 따른 투자매매업자나) 「자본시장과 금융투자업에 관한 법률」 제9조제19항제2호에 따른 투자중개업자다) 「자본시장과 금융투자업에 관한 법률」 제9조제19항제3호에 따른 투자일임업자라) 「자본시장과 금융투자업에 관한 법률」 제9조제19항제4호에 따른 투자신탁업자마) 「자본시장과 금융투자업에 관한 법률」 제9조제19항제5호에 따른 투자유한책임사원(투자유한책임사원)2) 신탁업자는 다음 각 목의 어느 하나에 해당하는 자를 말한다.가) 「자본시장과 금융투자업에 관한 법률」 제9조제19항제1호에 따른 투자매매업자나) 「자본시장과 금융'

In [ ]:
gen('1인 자영업자?')

'사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를 말한다.  1. 사업자등록을 한 1인이 상시적으로 고용하지 않고 독립적으로 사업을 영위하는 자를'